In [1]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

In [2]:
!pip install -q langchain

In [3]:
!pip install pypdf
!pip install rapidocr-onnxruntime
!pip install pdf2image
!pip install pdfminer
!pip install pdfminer-six

In [4]:
!pip install pypdfium2

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader




loader = PyPDFLoader("workspace/Robin/test_data/regles_MESBG.pdf")
#loader = PyPDFLoader("workspace/Robin/test_data/regles_MESBG.pdf")
docs = loader.load_and_split()

In [6]:
docs[0]

Document(page_content='1\nLe Grand Livre \nd e Règles\nd e la Terre d u Milieu', metadata={'source': 'workspace/Robin/test_data/regles_MESBG.pdf', 'page': 0})

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)

In [8]:
!pip install -U numpy

In [9]:
!export MKL_SERVICE_FORCE_INTEL=1

In [10]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

EMBEDDING_MODEL_NAME = "thenlper/gte-small"

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)
db = FAISS.from_documents(chunked_docs,embedding_model)

/home/linagora/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/linagora/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/linagora/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/linagora/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _

In [11]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [18]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=2000,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>

ChatGPT
Répondez à la question en fonction de vos connaissances. Utilisez le contexte suivant pour vous aider :
{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [19]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [23]:
question = "Quel sont les 4 phases de jeu du jeu Middle Earth Strategy Battle Game??"

In [24]:
llm_chain.invoke({"context": "", "question": question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" The Middle-earth Strategy Battle Game, also known as Middle-earth: Shadow of War, is a tactical action role-playing video game developed by Monolith Productions and published by Warner Bros. Interactive Entertainment. While the game does have a campaign mode with different phases, it doesn't strictly follow a four-phase structure like some other games might. However, we can identify some key aspects or stages in the gameplay experience. Here are some elements that could be considered phases, although they may not be strictly linear:\n\n  1. **Exploration and Conquest**: This phase involves exploring the open world of Middle-earth, discovering new areas, and conquering strongholds to expand your army and influence.\n   \n  2. **War**: Once you've taken control of a stronghold, you must defend it from enemy attacks. This includes managing resources, training your army, and leading them into battle against enemy forces.\n   \n  3. **Forge Alliances**: Building relationships with other c

In [25]:
rag_chain.invoke(question)

/home/linagora/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/linagora/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/linagora/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' The four phases of gameplay in Middle Earth Strategy Battle Game are as follows:\n  1. **Phase de Déplacement**: Each player moves their figures according to their movement capabilities. And when one player has finished, the other player can do the same.\n  2. **Phase de Tir**: Each player can make their figures with a shooting weapon shoot first, starting with the player who has initiative.\n  3. **Phase de Combat**: All engaged figures must fight. The player with initiative decides the order of resolution of combats.\n  4. **Phase de Fin**: All effects that end at the end of the turn are resolved, such as Paralysis, etc.'